# Step 0.1: Data Loading

In [1]:
import os
import pandas as pd
from openai import OpenAI
from func import get_user_watch_history
from sklearn.model_selection import train_test_split

client = OpenAI()

user_id = 123
candidate_size = 100

data = pd.read_csv("data/merged_df.csv")

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_user_df = train_data[train_data["userId"] == user_id]
train_title = train_user_df["title"]

test_user_df = test_data[test_data["userId"] == user_id]
test_title = test_user_df["title"]



In [3]:
from src.three_step import *

threeStepRecommender = ThreeStepRecommender(user_id, train_data, candidate_size)
movie_pred = threeStepRecommender.get_pred()
accuracy = threeStepRecommender.accuracy(movie_pred, test_title)

NameError: name 'movie_rating' is not defined

In [224]:
from src.baseline import *

mostPopularRecommendor = MostPopularRecommendor(train_data)
baseline_pred = mostPopularRecommendor.pred()
print(accuracy(baseline_pred, test_title))

NameError: name 'train_data' is not defined

Results:

1. The GPT model gives results not in testing even if we specify a testing set. It only support a candidate set of around 100.
2. The GPT gives recommendation with accuracy 0.5, while the "most popular" recommender only gives recommendation with accuracy 0.2.